In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
import nltk

nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


## Challenge 0:

For first challenge, we need to try changing data frame and try if it can increase the score. So for that we'll increase the number of features by including part of speech, dependencies and number of words to each sentence.

Then we'll check if it increases the score for our models.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words, POS, DEP):
        
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words.union(POS).union(DEP))
    df['NOW'] = 0
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    df.loc[:, POS] = 0
    df.loc[:, DEP] = 0
     
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        words_pos = [token.pos_
                     for token in sentence
                     if (
                         not token.is_punct
                         and not token.is_stop
                     )]
        
        words_dep = [token.dep_
                     for token in sentence
                     if (
                         not token.is_punct
                         and not token.is_stop
                     )]
        
        # Populate the row with word counts.
        
        df.loc[i, 'NOW'] = len(words)
        
        for word in words:
            df.loc[i, word] += 1
            
        for word in words_pos:
            df.loc[i, word] += 1
            
        for word in words_dep:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
pos = set(['CCONJ', 'PROPN', 'VERB', 'ADP', 'NOUN', 'SYM', 'NUM','DET', 'ADJ', 'ADV', 'AUX', 'CONJ', 'INTJ', 'PART', 'PRON', 'PUNCT', 'SCONJ', 'X'])
dep = set(['ROOT', 'npadvmod', 'predet', 'nmod', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'cop', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nn', 'nounmod', 'npmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'obj', 'obl', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'root', 'xcomp'])
word_counts = bow_features(sentences, common_words, pos, dep)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,introduce,realize,twice,settle,charm,thirteen,anybody,furnish,fortitude,cake,...,heaven,beautifully,rigid,hole,add,antipathies,range,NOW,text_sentence,text_source
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20.0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,24.0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11.0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.0,"(I, shall, be, late, !, ')",Carroll


In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier(n_estimators=100)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.996

Test set score: 0.8383233532934131


In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2', solver='liblinear') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
cross_val_score(lr, X_train, y_train, cv=5)

(250, 1715) (250,)
Training set score: 0.992

Test set score: 0.874251497005988


array([0.84313725, 0.90196078, 0.76      , 0.79591837, 0.87755102])

In [9]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
print("Training set score: ", svc.score(X_train, y_train))
print("Test set score: ", svc.score(X_test, y_test))

C:\Users\vivek\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training set score:  0.688
Test set score:  0.6826347305389222


## Challenge 1:

Now we'll try some other text and try whether it works with our newly generated model or not.

In [10]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [11]:
ball = gutenberg.raw('chesterton-ball.txt')

# The Chapter indicator is idiosyncratic
ball = re.sub(r'Chapter \d+', '', ball)
    
ball = text_cleaner(ball[:int(len(ball)/9)])

In [12]:
ball_doc = nlp(ball)

In [13]:
ball_sents = [[sent, "Chesterton"] for sent in ball_doc.sents]

sentences = pd.DataFrame(ball_sents + alice_sents)
sentences.head()

,0,1
0,"(I., A, DISCUSSION, SOMEWHAT, IN, THE, AIR)",Chesterton
1,"(The, flying, ship, of, Professor, Lucifer, sa...",Chesterton
2,"(That, it, was, far, above, the, earth, was, n...",Chesterton
3,"(The, professor, had, himself, invented, the, ...",Chesterton
4,"(Every, sort, of, tool, or, apparatus, had, ,,...",Chesterton


In [14]:
ballwords = bag_of_words(ball_doc)

# Combine bags to create a set of unique words.
common_words = set(ballwords + alicewords)

In [15]:
word_counts = bow_features(sentences, common_words, pos, dep)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600


,realize,twice,cataract,cloudlet,anybody,plot,carelessness,physical,voiceless,cake,...,sharpen,heaven,firm,beautifully,hole,antipathies,beneath,NOW,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.0,"(I., A, DISCUSSION, SOMEWHAT, IN, THE, AIR)",Chesterton
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,18.0,"(The, flying, ship, of, Professor, Lucifer, sa...",Chesterton
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.0,"(That, it, was, far, above, the, earth, was, n...",Chesterton
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.0,"(The, professor, had, himself, invented, the, ...",Chesterton
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11.0,"(Every, sort, of, tool, or, apparatus, had, ,,...",Chesterton


In [16]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))
X = X[:, :1715]

In [17]:
print(lr.score(X, Y))

0.16822429906542055


# Conclusion

In conclusion, the accuracy of our model increased for data on which it was trained. But, when we added new data from gutenberg it's accuracy decreased drastically. Maybe it's because our model was overfitted.